<a href="https://colab.research.google.com/github/kmk4444/Huggingface/blob/main/Zero_shot_audio_classificat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 5: Zero-Shot Audio Classification

In [ ]:
!pip install transformers
!pip install datasets
!pip install soundfile
!pip install librosa

- Here is some code that suppresses warning messages.

In [10]:
from transformers.utils import logging
logging.set_verbosity_error()

### Prepare the dataset of audio recordings

In [11]:
from datasets import load_dataset, load_from_disk

# This dataset is a collection of different sounds of 5 seconds
dataset = load_dataset("ashraq/esc50",
                      split="train[0:10]")
#dataset = load_from_disk("./models/ashraq/esc50/train")

Repo card metadata block was not found. Setting CardData to empty.


In [12]:
audio_sample = dataset[0]

In [13]:
audio_sample

{'filename': '1-100032-A-0.wav',
 'fold': 1,
 'target': 0,
 'category': 'dog',
 'esc10': True,
 'src_file': 100032,
 'take': 'A',
 'audio': {'path': None,
  'array': array([0., 0., 0., ..., 0., 0., 0.]),
  'sampling_rate': 44100}}

In [ ]:
from IPython.display import Audio as IPythonAudio
IPythonAudio(audio_sample["audio"]["array"],
             rate=audio_sample["audio"]["sampling_rate"])

### Build the `audio classification` pipeline using 🤗 Transformers Library

In [ ]:
from transformers import pipeline

In [15]:
zero_shot_classifier = pipeline(
    task="zero-shot-audio-classification",
    model="laion/clap-htsat-unfused")

More info on [laion/clap-htsat-unfused](https://huggingface.co/laion/clap-htsat-unfused).

### Sampling Rate for Transformer Models
- How long does 1 second of high resolution audio (192,000 Hz) appear to the Whisper model (which is trained to expect audio files at 16,000 Hz)?

In [ ]:
- The 1 second of high resolution audio appears to the model as if it is 12 seconds of audio.


In [16]:
(1 * 192000) / 16000

12.0

- How about 5 seconds of audio?

In [17]:
(5 * 192000) / 16000

60.0

- 5 seconds of high resolution audio appears to the model as if it is 60 seconds of audio.

In [18]:
zero_shot_classifier.feature_extractor.sampling_rate

48000

In [19]:
audio_sample["audio"]["sampling_rate"]

44100

* Set the correct sampling rate for the input and the model.

In [20]:
from datasets import Audio
datsaset = dataset.cast_column(
    "audio",
    Audio(sampling_rate=48_000))

In [21]:
audio_sample = dataset[0]

In [22]:
audio_sample

{'filename': '1-100032-A-0.wav',
 'fold': 1,
 'target': 0,
 'category': 'dog',
 'esc10': True,
 'src_file': 100032,
 'take': 'A',
 'audio': {'path': None,
  'array': array([0., 0., 0., ..., 0., 0., 0.]),
  'sampling_rate': 44100}}

In [23]:
candidate_labels = ["Sound of a dog",
                    "Sound of vacuum cleaner"]

In [24]:
zero_shot_classifier(audio_sample["audio"]["array"],
                     candidate_labels=candidate_labels)

[{'score': 0.9997242093086243, 'label': 'Sound of a dog'},
 {'score': 0.00027583082555793226, 'label': 'Sound of vacuum cleaner'}]

In [25]:
candidate_labels = ["Sound of a child crying",
                    "Sound of vacuum cleaner",
                    "Sound of a bird singing",
                    "Sound of an airplane"]

In [28]:
print(audio_sample["audio"]["array"])

[0. 0. 0. ... 0. 0. 0.]


In [26]:
zero_shot_classifier(audio_sample["audio"]["array"],
                     candidate_labels=candidate_labels)

[{'score': 0.6457511782646179, 'label': 'Sound of a bird singing'},
 {'score': 0.15903766453266144, 'label': 'Sound of vacuum cleaner'},
 {'score': 0.1563412994146347, 'label': 'Sound of an airplane'},
 {'score': 0.038869865238666534, 'label': 'Sound of a child crying'}]